In [1]:
pip install pygame

In [ ]:
import pandas as pd
import cv2
import time
from datetime import datetime
import pygame
import os

# Initialize Pygame for sound
pygame.mixer.init()
alarm_sound = pygame.mixer.Sound('C:\\Users\\naman\\Downloads\\Motion_Detection_System-main\\Motion_Detection_System-main\\mixkit-security-facility-breach-alarm-994.mp3')

initial_state = None
motion_track_list = [None, None]
motion_time = []
data_frame = pd.DataFrame(columns=["Initial", "Final"])

video = cv2.VideoCapture(0)

# Motion sensitivity control (adjustable threshold)
motion_sensitivity = 30  # Initial threshold

# Folder to save frames during motion events
motion_frame_save_path = "motion_frames"
# Folder to save cut videos during motion events
video_frame_save_path = "video_frames"
# Folder to save the complete video
complete_video_path = "complete_video"

# Create the directory to save frames during motion events if it doesn't exist
if not os.path.exists(motion_frame_save_path):
    os.makedirs(motion_frame_save_path)

# Create the directory to save cut videos during motion events if it doesn't exist
if not os.path.exists(video_frame_save_path):
    os.makedirs(video_frame_save_path)

# Create the directory to save the complete video if it doesn't exist
if not os.path.exists(complete_video_path):
    os.makedirs(complete_video_path)

# Video writer for complete video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
complete_video = cv2.VideoWriter(f'{complete_video_path}/complete_video.avi', fourcc, 20.0, (640, 480))  # Adjust resolution as needed

# Video writer for motion event clips
motion_event_video = None
motion_event_start_time = None

while True:
    ret, cur_frame = video.read()
    if not ret:
        print("Error: Unable to read frame from the video source.")
        break
    
    var_motion = 0
    gray_image = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_image, (21, 21), 0)

    if initial_state is None:
        initial_state = gray_frame
        continue

    differ_frame = cv2.absdiff(initial_state, gray_frame)
    thresh_frame = cv2.threshold(differ_frame, motion_sensitivity, 255, cv2.THRESH_BINARY)[1]
    thresh_frame = cv2.dilate(thresh_frame, None, iterations=2)

    contours, _ = cv2.findContours(thresh_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) < 10000:
            continue
        var_motion = 1
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(cur_frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

        # Save the frame during motion events
        motion_frame_filename = f"{motion_frame_save_path}/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.jpg"
        if not cv2.imwrite(motion_frame_filename, cur_frame):
            print("Error saving frame during motion event")
        else:
            print(f"Frame saved during motion event: {motion_frame_filename}")

        # Start recording a short video clip
        if motion_event_video is None:
            motion_event_start_time = datetime.now()
            motion_event_video = cv2.VideoWriter(f"{video_frame_save_path}/motion_clip_{motion_event_start_time.strftime('%Y-%m-%d_%H-%M-%S')}.avi", fourcc, 20.0, (640, 480))
            
            # Play the alarm sound when motion starts
            pygame.mixer.Sound.play(alarm_sound)  

    if var_motion == 0 and motion_event_video is not None:
        # If motion stops, release the video writer and stop the alarm sound
        motion_event_video.release()
        motion_event_video = None
        pygame.mixer.Sound.stop(alarm_sound)

    # Write frame to complete video
    complete_video.write(cur_frame)

    motion_track_list.append(var_motion)
    motion_track_list = motion_track_list[-2:]

    if motion_track_list[-1] == 1 and motion_track_list[-2] == 0:
        motion_time.append(datetime.now())

    if motion_track_list[-1] == 0 and motion_track_list[-2] == 1:
        motion_time.append(datetime.now())

    cv2.imshow("Gray Frame", gray_frame)
    cv2.imshow("Difference Frame", differ_frame)
    cv2.imshow("Threshold Frame", thresh_frame)
    cv2.imshow("Color Frame", cur_frame)

    wait_key = cv2.waitKey(1)
    if wait_key == ord('m'):
        if var_motion == 1:
            motion_time.append(datetime.now())
        break

for a in range(0, len(motion_time), 2):
    data_frame = data_frame.append({"Initial": motion_time[a], "Final": motion_time[a + 1]}, ignore_index=True)

data_frame.to_csv(f"{complete_video_path}/EachMovement.csv")
video.release()
complete_video.release()
if motion_event_video is not None:
    motion_event_video.release()

# Close OpenCV windows if any are open
if cv2.getBuildInformation().find('Win') > 0:
    cv2.destroyAllWindows()
